In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
c_folder="../../data/calibration/"
d_folder="../../data/runData/"
r_n=307
p_folder="../../plots/run_"+str(r_n)+"/"

In [4]:
# helper functions
def choose_bar(df,layer, strip): #works
    df=df[df["strip"]==strip]
    df=df[df["layer"]==layer]
    return df

def import_data(calibration_folder, data_folder, run_number, is_it_pulsed=False): # works for both pulsed and not-pulsed
    
    pedestal_df=pd.read_csv(calibration_folder+"pedestal_fixed.csv", sep=',')
    mip_df=pd.read_csv(calibration_folder+"mip_fixed.csv", sep=',')
    
    if is_it_pulsed==True:
        run_df=pd.read_csv(data_folder+"run_"+str(run_number)+"_pulse.csv", sep=',')
        run_df.drop(["pf_spill", "pf_ticks"], axis=1, inplace=True) # we never need these
        run_df.drop(["toa_end0", "toa_end1"], axis=1, inplace=True) # unreliable guys, but we might need them in the future
    else:
        run_df=pd.read_csv(data_folder+"run_"+str(run_number)+".csv", sep=',')
        run_df.drop(["pf_spill", "pf_ticks"], axis=1, inplace=True)
        run_df.drop(["toa_end0", "toa_end1"], axis=1, inplace=True)
        
    return pedestal_df,mip_df,run_df

In [5]:
def drop_extra_pedestal_data(df):
    df.drop(["pedestal_end0", "pedestal_end1",
             "pedestal_per_time_sample_end0","pedestal_per_time_sample_end1"], axis=1, inplace=True)
    return df

In [6]:
def subtract_pedestals(df, is_it_pulsed=False):
    df["adc_sum_end0"]-=df["pedestal_end0"]
    df["adc_sum_end1"]-=df["pedestal_end1"]
    
    df["adc_max_end0"]-=df["pedestal_per_time_sample_end0"]
    df["adc_max_end1"]-=df["pedestal_per_time_sample_end1"]
    
    df["adc_mean_end0"]-=df["pedestal_per_time_sample_end0"]
    df["adc_mean_end1"]-=df["pedestal_per_time_sample_end1"]

    if is_it_pulsed==True:
        for i in range(8):
            adc_end0_col = f'adc_{i}_end0'
            adc_end1_col = f'adc_{i}_end1'
        
            df[adc_end0_col]-=df["pedestal_per_time_sample_end0"]
            df[adc_end1_col]-=df["pedestal_per_time_sample_end1"]
    
    return df

In [18]:
def convert_mip_to_MeV(run_df,mip_df, is_it_pulsed):
    merged_df=run_df.merge(mip_df, how='left', on=['layer','strip'])

    end0=merged_df.columns.str.contains("adc_.*_end0")
    end1=merged_df.columns.str.contains("adc_.*_end1")
    
    merged_df[merged_df.columns[end0]]=merged_df[merged_df.columns[end0]].mul(4.66/merged_df["mpv_end0"], axis=0)
    #print(merged_df.head())
    merged_df[merged_df.columns[end1]]=merged_df[merged_df.columns[end1]].mul(4.66/merged_df["mpv_end1"], axis=0)

    merged_df.drop(["mpv_end0", "mpv_end1"], axis=1, inplace=True)
    
    return merged_df

In [19]:
def select_bars_with_data(run_df, pedestal_df, mip_df=None, subtract_pedestal=False, convert_mip=False, is_it_pulsed=False):
    
    merged_df=run_df.merge(pedestal_df, how='left', on=['layer', 'strip'])

    merged_df=merged_df[(merged_df["adc_sum_end0"]>(merged_df["pedestal_end0"]+5*merged_df["std_dev_end0"])) & 
                        (merged_df["adc_sum_end1"]>(merged_df["pedestal_end1"]+5*merged_df["std_dev_end1"]))]
    
    merged_df.drop(["std_dev_end0", "std_dev_end1"], axis=1, inplace=True)

    if subtract_pedestal==True:
        merged_df=subtract_pedestals(merged_df, is_it_pulsed)

    merged_df=drop_extra_pedestal_data(merged_df) # needs to be dropped regardless of subtraction, too much memory used otherwise

    if convert_mip==True: #similarily
        merged_df=convert_mip_to_MeV(merged_df, mip_df, is_it_pulsed)
        
    return merged_df

In [20]:
p,m,r=import_data(c_folder,d_folder, 300, is_it_pulsed=True)

In [21]:
r=select_bars_with_data(r,p, mip_df=m, subtract_pedestal=True, convert_mip=True, is_it_pulsed=True) #this needs to be run as it's own cell

In [22]:
r.sample(10)

,pf_event,tot_end0,adc_sum_end0,adc_mean_end0,adc_max_end0,adc_0_end0,adc_1_end0,adc_2_end0,adc_3_end0,adc_4_end0,adc_5_end0,adc_6_end0,adc_7_end0,layer,strip,tot_end1,adc_sum_end1,adc_mean_end1,adc_max_end1,adc_0_end1,adc_1_end1,adc_2_end1,adc_3_end1,adc_4_end1,adc_5_end1,adc_6_end1,adc_7_end1,pedestal_per_time_sample_std_dev_end0,pedestal_per_time_sample_std_dev_end1
116135,31153,0,3.085930,0.385741,0.592346,-0.038216,0.296173,0.487252,0.573238,0.592346,0.525468,0.343943,0.305727,2,2,0,3.403141,0.425393,0.664753,0.019843,0.377024,0.535771,0.615145,0.664753,0.535771,0.357181,0.297651,1.276401,1.709174
967782,20208,341,17.188692,2.148587,5.224732,0.101342,3.135965,5.224732,5.162801,2.026836,0.754433,0.450408,0.332176,6,3,4024,16.699083,2.087385,6.356337,0.102743,1.767171,6.356337,4.554918,2.034302,0.876736,0.568509,0.438368,1.312134,1.197089
1754210,18898,0,3.062197,0.382775,0.655015,0.196505,0.450323,0.605889,0.655015,0.573139,0.376634,0.155566,0.049126,11,5,0,1.834345,0.229293,0.395643,0.143870,0.287740,0.388450,0.395643,0.345288,0.244579,0.071935,-0.043161,1.385571,1.870940
1361399,23484,0,4.596668,0.574584,0.934162,0.081554,0.378113,0.704328,0.934162,0.904506,0.793296,0.556049,0.244661,8,3,0,3.158167,0.394771,0.686254,0.035013,0.266098,0.490181,0.686254,0.686254,0.567210,0.315116,0.112041,1.283756,1.767760
998486,54827,3112,41.797855,5.224732,5.224732,5.224732,5.224732,5.224732,5.224732,5.224732,5.224732,5.224732,5.224732,6,3,344,49.049275,6.131159,6.356337,6.356337,6.356337,6.356337,6.356337,6.356337,6.356337,6.356337,4.554918,1.312134,1.197089
1006945,64065,1056,20.273986,2.534248,5.224732,0.202684,5.224732,5.224732,5.224732,2.938912,0.669982,0.439148,0.349066,6,3,520,24.165039,3.020630,6.356337,0.178087,6.356337,6.356337,6.356337,3.150770,0.821940,0.534261,0.410970,1.312134,1.197089
619145,18342,0,0.639000,0.079875,0.299106,-0.271915,-0.278713,-0.054383,0.203936,0.299106,0.265117,0.271915,0.203936,5,0,0,0.610051,0.076256,0.285556,-0.233637,-0.207677,0.000000,0.207677,0.285556,0.233637,0.207677,0.116818,2.099055,1.397454
715918,17067,0,33.435434,4.179429,5.614926,5.614926,5.614926,5.614926,5.614926,5.614926,5.614926,0.181517,-0.435641,5,3,0,36.867451,4.608431,6.233594,6.233594,6.233594,6.233594,6.233594,6.233594,6.233594,0.033805,-0.567920,1.252527,3.143165
171579,50256,0,4.379993,0.547499,1.069533,0.000000,0.040744,0.407441,0.845440,1.069533,0.937115,0.631534,0.448185,2,3,0,4.264870,0.533109,1.068691,0.029686,0.039581,0.395812,0.811414,1.068691,0.910367,0.593717,0.415602,1.861981,1.484994
666586,26125,0,4.395723,0.549465,1.410895,-0.012541,-0.025083,0.369968,1.053469,1.410895,0.771290,0.476569,0.351156,5,2,0,2.020528,0.252566,0.537888,0.000000,-0.151712,0.124128,0.337904,0.468928,0.537888,0.399968,0.303424,1.220176,2.132965
